##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 TFX 流水线和 TensorFlow Model Analysis 进行模型分析


注：我们建议在 Colab 笔记本中运行本教程，无需进行设置！只需点击“在 Google Colab 中运行”

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_tfma"> <img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/penguin_tfma.ipynb"> <img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/penguin_tfma.ipynb"> <img width="32px" src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/tfx/penguin_tfma.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载此 notebook</a></td>
</table></div>

在这个基于笔记本的教程中，我们将创建和运行一个 TFX 流水线，该流水线会创建一个简单的分类模型并分析其在多次运行中的性能。此笔记本基于我们在[简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple)中构建的 TFX 流水线。如果您尚未阅读该教程，请先阅读，然后再继续此笔记本。

当您调整模型或使用新数据集训练模型时，您需要检查模型有所改进还是变得更差。仅检查准确率等顶级指标可能还不够。每个经过训练的模型在投入生产之前都应进行评估。

我们将向上一个教程中创建的流水线添加一个 `Evaluator` 组件。Evaluator 组件会对您的模型进行深入分析，并将新模型与基准进行比较，以确定它们“足够好”。它使用 [TensorFlow 模型分析](https://tensorflow.google.cn/tfx/guide/tfma)库实现。

要详细了解 TFX 中的各种概念，请参阅[了解 TFX 流水线](https://tensorflow.google.cn/tfx/guide/understanding_tfx_pipelines)。

## 安装

安装过程与上一个教程相同。

我们首先需要安装 TFX Python 软件包并下载将用于模型的数据集。

### 升级 Pip

为了避免在本地运行时升级系统中的 Pip，请检查以确保在 Colab 中运行。当然，可以对本地系统单独升级。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### 安装 TFX


In [ ]:
!pip install -U tfx

### 卸载 Shapely

TODO(b/263441833) 这是避免 ImportError 的临时解决方案。最终，应该通过支持最新版本的 Bigquery 来处理，而不是卸载其他额外的依赖项。

In [ ]:
!pip uninstall shapely -y

### 是否已重新启动运行时？

如果您使用的是 Google Colab，则在首次运行上面的代码单元时必须重新启动运行时，方法是点击上面的“RESTART RUNTIME”按钮或使用“Runtime &gt; Restart runtime ...”菜单。这样做的原因是 Colab 加载软件包的方式。

检查 TensorFlow 和 TFX 版本。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### 设置变量

有一些变量用于定义流水线。您可以根据需要自定义这些变量。默认情况下，流水线的所有输出都将在当前目录下生成。

In [ ]:
import os

PIPELINE_NAME = "penguin-tfma"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### 准备示例数据

我们使用相同的 [Palmer Penguins 数据集](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)。

此数据集中有四个数字特征，这些特征已标准化为具有范围 [0,1]。我们将建立一个预测企鹅 `species` 的分类模型。

因为 TFX ExampleGen 从目录中读取输入，所以我们需要创建一个目录并将数据集复制到其中。

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

## 创建流水线

我们将向在[简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple)中创建的流水线中添加一个 [`Evaluator`](https://tensorflow.google.cn/tfx/guide/evaluator) 组件。

Evaluator 组件需要来自 `ExampleGen` 组件的输入数据、来自 `Trainer` 组件的模型，以及一个 [`tfma.EvalConfig`](https://tensorflow.google.cn/tfx/model_analysis/api_docs/python/tfma/EvalConfig) 对象。我们可以选择提供一个基准模型，该模型可用于将指标与新训练的模型进行比较。

Evaluator 会创建两种输出工件：`ModelEvaluation` 和 `ModelBlessing`。ModelEvaluation 包含详细的评估结果，可以使用 TFMA 库对该结果进一步研究和可视化。ModelBlessing 包含一个布尔结果，表示模型是否传递了给定的标准，并且可以在以后的组件（如 Pusher）中用作信号。


### 编写模型训练代码

我们将使用与[简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple)中相同的模型代码。

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://tensorflow.google.cn/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

### 编写流水线定义

我们将定义一个函数来创建 TFX 流水线。除了上面提到的 Evaluator 组件之外，我们还将添加一个名为 [`Resolver`](https://tensorflow.google.cn/tfx/api_docs/python/tfx/v1/dsl/Resolver) 的节点。为了检查新模型是否比以前的模型更好，我们需要将其与以前发布的模型（称为基准）进行比较。[ML Metadata (MLMD)](https://tensorflow.google.cn/tfx/guide/mlmd) 会跟踪流水线的所有先前工件，而 `Resolver` 可以使用名为 `LatestBlessedModelStrategy` 的策略类从 MLMD 找到最新的*受祝福*模型（成功传递 Evaluator 的模型）。


In [ ]:
import tensorflow_model_analysis as tfma

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # NEW: Get the latest blessed model for Evaluator.
  model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

  # NEW: Uses TFMA to compute evaluation statistics over features of a model and
  #   perform quality validation of a candidate model (compared to a baseline).

  eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='species')],
      slicing_specs=[
          # An empty slice spec means the overall slice, i.e. the whole dataset.
          tfma.SlicingSpec(),
          # Calculate metrics for each penguin species.
          tfma.SlicingSpec(feature_keys=['species']),
          ],
      metrics_specs=[
          tfma.MetricsSpec(per_slice_thresholds={
              'sparse_categorical_accuracy':
                  tfma.PerSliceMetricThresholds(thresholds=[
                      tfma.PerSliceMetricThreshold(
                          slicing_specs=[tfma.SlicingSpec()],
                          threshold=tfma.MetricThreshold(
                              value_threshold=tfma.GenericValueThreshold(
                                   lower_bound={'value': 0.6}),
                              # Change threshold will be ignored if there is no
                              # baseline model resolved from MLMD (first run).
                              change_threshold=tfma.GenericChangeThreshold(
                                  direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                  absolute={'value': -1e-10}))
                       )]),
          })],
      )
  evaluator = tfx.components.Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      eval_config=eval_config)

  # Checks whether the model passed the validation steps and pushes the model
  # to a file destination if check passed.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      model_blessing=evaluator.outputs['blessing'], # Pass an evaluation result.
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      trainer,

      # Following two components were added to the pipeline.
      model_resolver,
      evaluator,

      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

我们需要通过 `eval_config` 向 Evaluator 提供以下信息：

- 要配置的其他指标（如果需要比模型中定义的指标更多的指标）。
- 要配置的切片
- 用于确认是否包含验证的模型验证阈值

由于 `SparseCategoricalAccuracy` 已包含在 `model.compile()` 调用中，它将自动包含在分析中。因此，我们在这里不添加任何额外的指标。`SparseCategoricalAccuracy` 也将用于决定模型是否足够好。

我们计算整个数据集和每个企鹅物种的指标。`SlicingSpec` 指定我们如何聚合声明的指标。

新模型需要传递两个阈值，一个是绝对阈值0.6，另一个是相对阈值，它应该高于基准模型。当您第一次运行流水线时，`change_threshold` 会被忽略，仅会检查 value_threshold。如果您多次运行流水线，`Resolver` 将找到上次运行的模型，并将其用作进行比较的基准模型。

如需了解更多信息，请参阅 [Evaluator 组件指南](https://tensorflow.google.cn/tfx/guide/evaluator#using_the_evaluator_component)。

## 运行流水线


我们将像上一教程一样使用 `LocalDagRunner`。

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

流水线完成后，您应该能够看到如下内容：

```
INFO:absl:Blessing result True written to pipelines/penguin-tfma/Evaluator/blessing/4.
```

或者，您也可以手动检查存储已生成工件的输出目录。如果使用文件浏览器访问 `pipelines/penguin-tfma/Evaluator/blessing/`，根据评估结果，您可以看到名称为 `BLESSED` 或 `NOT_BLESSED` 的文件。

如果祝福结果为 `False`，Pusher 会拒绝将模型推送到 `serving_model_dir`，因为该模型不够好，无法在生产中使用。

您可以使用不同的评估配置再次运行流水线。即使您使用完全相同的配置和数据集运行流水线，由于模型训练固有的随机性，训练后的模型也可能会略有不同，这可能会导致 `NOT_BLESSED` 模型。

### 检查流水线的输出

您可以使用 TFMA 来调查和可视化 ModelEvaluation 工件中的评估结果。

> **注：如果您不在 Colab 上，请安装 Jupyter 扩展程序**。您需要 TensorFlow Model Analysis 扩展程序才能查看 TFMA 的可视化效果。此扩展程序已安装在 Google Colab 上，但如果您在其他环境中运行此笔记本，则可能需要安装该扩展程序。请参阅[安装指南](https://github.com/tensorflow/model-analysis#installation)中 Jupyter 扩展程序的安装方向。


#### 从输出工件中获取分析结果

您可以使用 MLMD API 以编程方式定位这些输出。首先，我们将定义一些效用函数来搜索刚刚生成的输出工件。

In [ ]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_output_artifacts(metadata, latest_execution.id)


我们可以找到 `Evaluator` 组件的最新执行并获得它的输出工件。

In [ ]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  evaluator_output = get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                          'Evaluator')
  eval_artifact = evaluator_output[standard_component_specs.EVALUATION_KEY][0]

`Evaluator` 始终返回一个评估工件，我们可以使用 TensorFlow Model Analysis 库将其可视化。例如，以下代码将呈现每个企鹅物种的准确率指标。

In [ ]:
import tensorflow_model_analysis as tfma

eval_result = tfma.load_eval_result(eval_artifact.uri)
tfma.view.render_slicing_metrics(eval_result, slicing_column='species')

如果您在 `Show` 下拉列表中选择 'sparse_categorical_accuracy'，则可以看到每个物种的准确率值。您可能想要添加更多切片并检查您的模型是否适合所有分布以及是否存在任何可能的偏差。

## 后续步骤

要详细了解模型分析，请参阅 [TensorFlow Model Analysis 库教程](https://tensorflow.google.cn/tfx/tutorials/model_analysis/tfma_basic)。

您可以在 https://tensorflow.google.cn/tfx/tutorials 上找到更多资源。

请参阅[了解 TFX 流水线](https://tensorflow.google.cn/tfx/guide/understanding_tfx_pipelines)来详细了解 TFX 中的各种概念。
